**HYBRID WATERMARK EMBEDDING**

In [5]:
pip install PyWavelets


In [6]:
# hybrid_dwt_dct_embed_color.py
import cv2
import numpy as np
import pywt

def hybrid_dwt_dct_embed_color(cover_path, watermark_path, output_path, alpha=0.1):
    # Step 1: Read images
    cover = cv2.imread(cover_path)
    watermark = cv2.imread(watermark_path, cv2.IMREAD_GRAYSCALE)
    if cover is None or watermark is None:
        raise FileNotFoundError("Check your image paths!")

    # Step 2: Resize images
    cover = cv2.resize(cover, (512, 512))
    watermark = cv2.resize(watermark, (128, 128))

    # Step 3: Convert to YCrCb and split
    ycrcb = cv2.cvtColor(cover, cv2.COLOR_BGR2YCrCb)
    Y, Cr, Cb = cv2.split(ycrcb)

    # Step 4: Apply DWT on Y channel
    LL, (LH, HL, HH) = pywt.dwt2(np.float32(Y), 'haar')

    # Step 5: Apply DCT on LL
    dct_LL = cv2.dct(LL)

    # Step 6: Normalize watermark
    watermark_norm = np.float32(watermark) / 255.0

    # Step 7: Embed watermark
    dct_LL[0:128, 0:128] += alpha * watermark_norm

    # Step 8: Inverse DCT and DWT
    LL_new = cv2.idct(dct_LL)
    Y_new = pywt.idwt2((LL_new, (LH, HL, HH)), 'haar')
    Y_new = np.clip(Y_new, 0, 255).astype(np.uint8)

    # Step 9: Merge and save color image
    watermarked_ycrcb = cv2.merge((Y_new, Cr, Cb))
    watermarked_bgr = cv2.cvtColor(watermarked_ycrcb, cv2.COLOR_YCrCb2BGR)
    cv2.imwrite(output_path, watermarked_bgr)

    print(f"Watermark embedded successfully → {output_path}")

if __name__ == "__main__":
    hybrid_dwt_dct_embed_color(
        cover_path="cover.jpeg",
        watermark_path="logo.png",
        output_path="watermarked.png",
        alpha=0.1
    )


Watermark embedded successfully → watermarked.png


**HYBRID DETECTION**

In [8]:
# hybrid_dwt_dct_detect_color.py
import cv2
import numpy as np
import pywt

def hybrid_dwt_dct_detect_color(cover_path, watermarked_path, watermark_path, extracted_path, alpha=0.1, threshold=0.25):
    # Step 1: Load images
    cover = cv2.imread(cover_path)
    watermarked = cv2.imread(watermarked_path)
    watermark = cv2.imread(watermark_path, cv2.IMREAD_GRAYSCALE)
    if cover is None or watermarked is None or watermark is None:
        raise FileNotFoundError("Check your file paths!")

    # Step 2: Resize
    cover = cv2.resize(cover, (512, 512))
    watermarked = cv2.resize(watermarked, (512, 512))
    watermark = cv2.resize(watermark, (128, 128))

    # Step 3: Extract Y channels
    Y_cover = cv2.cvtColor(cover, cv2.COLOR_BGR2YCrCb)[:, :, 0]
    Y_watermarked = cv2.cvtColor(watermarked, cv2.COLOR_BGR2YCrCb)[:, :, 0]

    # Step 4: Apply DWT on both
    LL_c, (LH_c, HL_c, HH_c) = pywt.dwt2(np.float32(Y_cover), 'haar')
    LL_w, (LH_w, HL_w, HH_w) = pywt.dwt2(np.float32(Y_watermarked), 'haar')

    # Step 5: Apply DCT
    dct_LL_c = cv2.dct(LL_c)
    dct_LL_w = cv2.dct(LL_w)

    # Step 6: Extract watermark
    extracted = (dct_LL_w[0:128, 0:128] - dct_LL_c[0:128, 0:128]) / alpha
    extracted = np.clip(extracted, 0, 1)
    extracted = (extracted * 255).astype(np.uint8)

    # Step 7: Save extracted watermark
    cv2.imwrite(extracted_path, extracted)
    print(f"Extracted watermark saved → {extracted_path}")

    # Step 8: Compute correlation
    extracted_resized = cv2.resize(extracted, (128, 128))
    watermark = cv2.resize(watermark, (128, 128))

    extracted_norm = (extracted_resized - np.mean(extracted_resized)) / (np.std(extracted_resized) + 1e-6)
    watermark_norm = (watermark - np.mean(watermark)) / (np.std(watermark) + 1e-6)
    correlation = np.mean(extracted_norm * watermark_norm)

    if correlation > threshold:
        print(f"Watermark detected! Correlation = {correlation:.3f}")
    else:
        print(f"Watermark not detected. Correlation = {correlation:.3f}")

if __name__ == "__main__":
    hybrid_dwt_dct_detect_color(
        cover_path="cover.jpeg",
        watermarked_path="watermarked.png",
        watermark_path="logo.png",
        extracted_path="extracted.png",
        alpha=0.1,
        threshold=0.25
    )


Extracted watermark saved → extracted.png
Watermark detected! Correlation = 0.290
